In [90]:
import numpy as np

In [91]:
images_url = "./CUB_200_2011/images.txt"
train_test_split_url = "./CUB_200_2011/train_test_split.txt"
classes_url = "./CUB_200_2011/classes.txt"
image_class_labels_url = "./CUB_200_2011/image_class_labels.txt"

Loading .txt file

In [92]:
image = np.genfromtxt(images_url , delimiter=' ', dtype=str) #<image_id> <image_name>
train_test_split = np.genfromtxt(train_test_split_url, delimiter=" ", dtype=str) #<image_id> <is_training_image>
classes = np.genfromtxt(classes_url, delimiter=" ", dtype=str) #<class_id> <class_name>
image_class_labels = np.genfromtxt(image_class_labels_url, delimiter=" ", dtype=str) #<image_id> <class_id>

<table>
    <tr>
        <td>image</td>
        <td>&lt;image_id&gt; &lt;image_name&gt;</td>
    </tr>
    <tr>
        <td>train_test_split</td>
        <td>&lt;image_id&gt; &lt;is_training_image&gt;</td>
    </tr>
    <tr>
        <td>classes</td>
        <td>&lt;class_id&gt; &lt;class_name&gt;</td>
    </tr>
    <tr>
        <td>image_class_labels</td>
        <td>&lt;image_id&gt; &lt;class_id&gt;</td>
    </tr>
</table>

In [93]:
print(image[0])
print(train_test_split[0])
print(classes[0])
print(image_class_labels[0])

['1' '001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg']
['1' '0']
['1' '001.Black_footed_Albatross']
['1' '1']


In [109]:
image_train = train_test_split[train_test_split[:,1]=="1"]
image_test = train_test_split[train_test_split[:,1]=="0"]
n_train = image_train.shape[0]
n_test = image_test.shape[0]
n_classes = classes.shape[0]

print("Number of Images:", image.shape[0])
print(f"n_train: {n_train}")
print(f"n_test: {n_test}")
print(f"n_classes: {n_classes}")

Number of Images: 11788
n_train: 5994
n_test: 5794
n_classes: 200
